In [ ]:
import random
from collections import OrderedDict
import numpy as np

class YahtzeeGame:
    def __init__(self):
        self.categories = OrderedDict([
            ('Ones', None), ('Twos', None), ('Threes', None), ('Fours', None),
            ('Fives', None), ('Sixes', None), ('Three of a Kind', None),
            ('Four of a Kind', None), ('Full House', None), ('Small Straight', None),
            ('Large Straight', None), ('Yahtzee', None), ('Chance', None)
        ])
        self.upper_bonus = 0
        self.yahtzee_bonuses = 0
        self.dice = [0]*5
        self.rolls_left = 0

    def reset(self):
        for cat in self.categories:
            self.categories[cat] = None
        self.upper_bonus = 0
        self.yahtzee_bonuses = 0
        self.dice = [0]*5
        self.rolls_left = 0
        return self.get_state()

    def get_state(self):
        return {
            'categories': self.categories.copy(),
            'dice': self.dice.copy(),
            'rolls_left': self.rolls_left,
            'upper_bonus': self.upper_bonus,
            'yahtzee_bonuses': self.yahtzee_bonuses
        }

    def set_state(self, state):
        self.categories = state['categories'].copy()
        self.dice = state['dice'].copy()
        self.rolls_left = state['rolls_left']
        self.upper_bonus = state['upper_bonus']
        self.yahtzee_bonuses = state['yahtzee_bonuses']

    def roll_dice(self, keep_mask=None):
        if keep_mask is None:
            self.dice = [random.randint(1,6) for _ in range(5)]
        else:
            for i in range(5):
                if not keep_mask[i]:
                    self.dice[i] = random.randint(1,6)
        # self.dice.sort()
        self.rolls_left -= 1

    def get_possible_moves(self):
        return [cat for cat, score in self.categories.items() if score is None]

    def calculate_score(self, category, dice):
        counts = [dice.count(i) for i in range(1,7)]
        if category in ['Ones', 'Twos', 'Threes', 'Fours', 'Fives', 'Sixes']:
            value = int(category[0]) if category[0].isdigit() else 6
            return sum(d for d in dice if d == value)
        elif category == 'Three of a Kind':
            return sum(dice) if max(counts) >= 3 else 0
        elif category == 'Four of a Kind':
            return sum(dice) if max(counts) >= 4 else 0
        elif category == 'Full House':
            return 25 if (3 in counts and 2 in counts) else 0
        elif category == 'Small Straight':
            return 30 if any(all(x in dice for x in [i,i+1,i+2,i+3]) for i in [1,2,3]) else 0
        elif category == 'Large Straight':
            return 40 if any(all(x in dice for x in [i,i+1,i+2,i+3,i+4]) for i in [1,2]) else 0
        elif category == 'Yahtzee':
            return 50 if all(d == dice[0] for d in dice) else 0
        elif category == 'Chance':
            return sum(dice)
        return 0

    def apply_move(self, category):
        score = self.calculate_score(category, self.dice)
        
        if category in ['Ones', 'Twos', 'Threes', 'Fours', 'Fives', 'Sixes']:
            # Convert items to list before slicing
            upper_section = list(self.categories.items())[:6]
            upper_total = sum(v for k, v in upper_section if v is not None)
            if upper_total + score >= 63 and self.upper_bonus == 0:
                self.upper_bonus = 35
                
        if category == 'Yahtzee' and score == 50 and self.categories['Yahtzee'] is not None:
            self.yahtzee_bonuses += 100
            
        self.categories[category] = score
        return score

class YahtzeeAgent:
    def __init__(self):
        pass

    def choose_dice_to_keep(self, current_dice, rolls_left):
        return [random.randint(0,1) for _ in range(5)]  # Placeholder: keep all dice

    def choose_category(self, game_state):
        available = [cat for cat, score in game_state['categories'].items() if score is None]
        return random.choice(available)  # Placeholder: random choice

def simulation_mode(agent, num_games=1):
    for _ in range(num_games):
        game = YahtzeeGame()
        game.reset()
        turn_counter = 1
        cumulative_score = 0
        
        while None in game.categories.values():
            print(f"\n=== Turn {turn_counter} ===")
            game.rolls_left = 2
            game.roll_dice()
            
            for _ in range(2):
                print(f"\nDice: {game.dice}")
                keep_mask = agent.choose_dice_to_keep(game.dice, game.rolls_left)
                game.roll_dice(keep_mask)
                print(f"Kept: {[d for d, keep in zip(game.dice, keep_mask) if keep]}")
                
            print(f"\nFinal dice: {game.dice}")
            category = agent.choose_category(game.get_state())
            score = game.apply_move(category)
            cumulative_score = sum(v for v in game.categories.values() if v is not None)
            cumulative_score += game.upper_bonus + game.yahtzee_bonuses
            
            print(f"Chose category: {category} (Score: {score})")
            print("Current scores:", {k:v for k,v in game.categories.items() if v is not None})
            print(f"Total Score: {cumulative_score}")
            print("-------------------")
            turn_counter += 1
        
        print(f"\nGame Over! Final Score: {cumulative_score}\n")

import itertools

def all_possible_keep_patterns(dice):
    """Generate all unique value-based keep patterns for current dice"""
    patterns = set()
    counts = {num: dice.count(num) for num in set(dice)}
    
    # Generate possible value combinations
    for num in counts:
        for keep_count in range(counts[num] + 1):
            if keep_count == 0:
                continue
            # Generate mask for this value count
            pattern = []
            kept = 0
            for d in dice:
                if d == num and kept < keep_count:
                    pattern.append(True)
                    kept += 1
                else:
                    pattern.append(False)
            patterns.add(tuple(pattern))
    
    # Add combinations with multiple numbers
    for combo in itertools.combinations(set(dice), 2):
        for count1 in range(1, dice.count(combo[0]) + 1):
            for count2 in range(1, dice.count(combo[1]) + 1):
                pattern = []
                kept1 = kept2 = 0
                for d in dice:
                    if d == combo[0] and kept1 < count1:
                        pattern.append(True)
                        kept1 += 1
                    elif d == combo[1] and kept2 < count2:
                        pattern.append(True)
                        kept2 += 1
                    else:
                        pattern.append(False)
                patterns.add(tuple(pattern))
    
    return [list(p) for p in patterns]

def calculation_mode(agent):
    game = YahtzeeGame()
    
    # Get input with validation
    try:
        dice = list(map(int, input("Enter dice values (space-separated, 5 numbers): ").split()))
        assert len(dice) == 5, "Must enter exactly 5 dice values"
        used = input("Enter used categories (comma-separated): ").split(',')
        rolls_left = int(input("Enter rolls remaining (0-2): "))
        turn = int(input("Enter current turn (1-13): "))
    except Exception as e:
        print(f"Invalid input: {e}")
        return

    # Set game state
    game.dice = dice
    game.rolls_left = rolls_left
    for cat in used:
        if cat.strip():
            game.categories[cat.strip()] = 0
    
    # Get predictions
    print(f"\nAnalysis for Turn {turn} with {rolls_left} rolls remaining:")
    available = game.get_possible_moves()
    
    # Get keep patterns
    print("\nDice Keeping Recommendations:")
    patterns = all_possible_keep_patterns(dice)
    for pattern in patterns:
        keep_str = "".join(["K" if k else "_" for k in pattern])
        kept = [d for d, k in zip(dice, pattern) if k]
        
        # Calculate value-based pattern identifier
        value_counts = {}
        for num in kept:
            value_counts[num] = value_counts.get(num, 0) + 1
        pattern_id = "+".join(f"{v}x{k}" for k,v in sorted(value_counts.items()))
        
        # Placeholder evaluation score
        eval_score = random.uniform(0, 50)
        print(f"{keep_str} ({pattern_id}): {eval_score:.1f}")
    
    # Show category predictions
    print("\nCategory Predictions:")
    for move in available:
        q_value = random.uniform(0, 50)  # Placeholder
        immediate = game.calculate_score(move, dice)
        upper_bonus_impact = ""
        
        if move in ['Ones', 'Twos', 'Threes', 'Fours', 'Fives', 'Sixes']:
            current_upper = sum(v for k,v in list(game.categories.items())[:6] if v is not None)
            potential_upper = current_upper + immediate
            upper_bonus_impact = f" (Upper Bonus: {'+' if potential_upper >=63 else '-'})"
            
        print(f"{move:<20} | Q: {q_value:5.1f} | Now: {immediate:3d}{upper_bonus_impact}")
        
def performance_mode(agent, num_games=100):
    scores = []
    for _ in range(num_games):
        game = YahtzeeGame()
        game.reset()
        
        # Process exactly 13 turns (one per category)
        for _ in range(13):
            game.rolls_left = 2
            game.roll_dice()  # Initial roll
            
            # Perform two re-rolls
            for _ in range(2):
                keep_mask = agent.choose_dice_to_keep(game.dice, game.rolls_left)
                game.roll_dice(keep_mask)
                
            # Get available categories (should never be empty here)
            available = game.get_possible_moves()
            if not available:
                break  # Safety check
                
            category = agent.choose_category(game.get_state())
            game.apply_move(category)
        
        total = sum(v for v in game.categories.values() if v is not None)
        total += game.upper_bonus + game.yahtzee_bonuses
        scores.append(total)
    
    print(f"\nPerformance over {num_games} games:")
    print(f"Mean score: {np.mean(scores):.1f}")
    print(f"Median score: {np.median(scores):.1f}")

if __name__ == "__main__":
    agent = YahtzeeAgent()
    
    # while True:
    print("\nSelect mode:")
    print("1. Simulation Mode")
    print("2. Calculation Mode")
    print("3. Performance Mode")
    print("4. Exit")
    choice = input("Enter choice: ")
    
    if choice == '1':
        simulation_mode(agent)
    elif choice == '2':
        calculation_mode(agent)
    elif choice == '3':
        performance_mode(agent)
    # elif choice == '4':
    #     break
    else:
        print("Invalid choice")

In [ ]:
import yahtzee


class YahtzeeAgent:
    def __init__(self):
        pass

    def choose_dice_to_keep(self, current_dice, rolls_left):
        return [random.randint(0,1) for _ in range(5)]  # Placeholder: keep all dice

    def choose_category(self, game_state):
        available = [cat for cat, score in game_state['categories'].items() if score is None]
        return random.choice(available)  # Placeholder: random choice
    
    
agent = yahtzee.YahtzeeAgent()
simulation_mode(agent)






=== Turn 1 ===

Dice: [2, 4, 4, 5, 5]
Kept: [1, 2, 4]

Dice: [1, 2, 2, 4, 5]
Kept: [4, 5]

Final dice: [2, 3, 4, 4, 5]
Chose category: Sixes (Score: 0)
Current scores: {'Sixes': 0}
Total Score: 0
-------------------

=== Turn 2 ===

Dice: [1, 3, 3, 4, 6]
Kept: [3, 3, 3]

Dice: [2, 3, 3, 3, 4]
Kept: [4, 4]

Final dice: [3, 4, 4, 4, 4]
Chose category: Threes (Score: 0)
Current scores: {'Threes': 0, 'Sixes': 0}
Total Score: 0
-------------------

=== Turn 3 ===

Dice: [2, 2, 3, 4, 5]
Kept: [5]

Dice: [1, 4, 4, 5, 6]
Kept: [5]

Final dice: [1, 1, 1, 5, 5]
Chose category: Fours (Score: 0)
Current scores: {'Threes': 0, 'Fours': 0, 'Sixes': 0}
Total Score: 0
-------------------

=== Turn 4 ===

Dice: [1, 1, 1, 2, 3]
Kept: [6]

Dice: [1, 3, 5, 6, 6]
Kept: [6]

Final dice: [1, 1, 3, 6, 6]
Chose category: Fives (Score: 12)
Current scores: {'Threes': 0, 'Fours': 0, 'Fives': 12, 'Sixes': 0}
Total Score: 12
-------------------

=== Turn 5 ===

Dice: [1, 3, 3, 4, 4]
Kept: [5]

Dice: [1, 2, 2, 4, 5]

In [2]:
[bool(int(bit)) for bit in f"{0:03b}"]


[False, False, False]